# Data operations for prepping the WorldBank_Health Dataset 

### Summary: 
The dataset [`world_bank_health_population`](https://cloud.google.com/bigquery?sq=1057666841514:3bb229234d7f4b379098581f0101e923&_ga=2.81369303.-1379782407.1673021064&project=paulkamau&ws=!1m9!1m3!3m2!1sbigquery-public-data!2sworld_bank_health_population!1m4!4m3!1sbigquery-public-data!2sworld_bank_health_population!3scountry_series_definitions) combines key health statistics from a variety of sources to provide a look at global health and population trends. It includes information on nutrition, reproductive health, education, immunization, and diseases from over 200 countries.


Multiple tables have been used to compile the data. The data was flattened to create a single working set: 

- Country series definitions 
- Country summary 
- Health nutrition population
- International debt
- Series summary 
- Series times

### Actions: 
1. Created a pivot table which takes the indicator codes as rows and tranforms them into columns in a new table. 
1. Created a merged table of the countries_data and the new pivot table we made
1. Created a table definition of the indicator_code and their description

### 1. Pivot Query. Convert the row values into column values 

<!-- SQL to create a pivot table which reates column data out of the row data from internation_debt table -->

original table we want: 
  `bigquery-public-data.world_bank_intl_debt.international_debt`

New pivot table with everything: 
--paulkamau.WorldBank_Debt.international_debt_pivot

```sql

DECLARE IndicatorCodes STRING;

SET IndicatorCodes = (
  SELECT 
    CONCAT('("', STRING_AGG(DISTINCT replace(indicator_code,".","_"), '", "'), '")'),
  FROM `bigquery-public-data.world_bank_intl_debt.international_debt` where value is not null
);


EXECUTE IMMEDIATE format("""
CREATE TABLE IF NOT EXISTS `paulkamau.WorldBank_Debt.international_debt_pivot` as 
SELECT * FROM
(
  SELECT country_name, country_code, replace(indicator_code,".","_") as ic,value
  FROM `bigquery-public-data.world_bank_intl_debt.international_debt`
) 
PIVOT
(
  MAX(value)
  FOR ic in %s
)
ORDER BY country_name ASC
""",
IndicatorCodes);

```

## Create the country_data table 

```sql

CREATE TABLE IF NOT EXISTS `paulkamau.WorldBank_Debt.countries_data` as 
SELECT 
country_code, short_name, currency_unit, income_group, national_accounts_base_year, sna_price_valuation, system_of_national_accounts, system_of_trade, government_accounting_concept
 FROM `bigquery-public-data.world_bank_intl_debt.country_summary`
where system_of_trade is not null
LIMIT 1000;

```
